# Gaussian Process Regression - HepG2 Cryoprotectant Optimization

Otimização de crioprotetores (DMSO e Trehalose) para maximizar viabilidade celular usando Gaussian Process.

## Importar bibliotecas e configurar constantes

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [6]:
BASE_DIR = Path('..').resolve()
FEATURES = ['% DMSO', 'TREHALOSE']
TARGET = '% QUEDA DA VIABILIDADE'

## Carregar e preparar dados

Leitura do CSV com conversão de valores percentuais e decimais em virgula. Limpeza de valores ausentes e combinações inválidas.

In [7]:
def safe_float(x):
    s = str(x).replace('%', '').replace(',', '.').strip()
    return float('nan') if s in ('', 'nan') else float(s)

df = pd.read_csv(BASE_DIR / 'data/raw/hepg2.csv', decimal=',', thousands='.')

for col in FEATURES + [TARGET]:
    df[col] = df[col].apply(safe_float)

df = df.dropna(subset=FEATURES + [TARGET])
df = df[~((df[FEATURES[0]] == 0) & (df[FEATURES[1]] == 0))]

df = df[((df[FEATURES] >= 0).all(axis=1)) & ((df[FEATURES] <= 100).all(axis=1))]

print(f"Dataset: {len(df)} samples")
print(f"Viability drop: {df[TARGET].min():.2f}% - {df[TARGET].max():.2f}% (mean: {df[TARGET].mean():.2f}%)")

Dataset: 200 samples
Viability drop: 0.15% - 100.00% (mean: 45.10%)


## Preparar features e normalizar dados

Divisão em conjuntos treino/teste e normalização com StandardScaler.

In [8]:
X = df[FEATURES].values
y = df[TARGET].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} | Test set: {X_test.shape[0]}")

Training set: 160 | Test set: 40


## Treinar modelo e avaliar performance

Gaussian Process com kernel RBF + WhiteKernel. Cálculo de R², MAE, RMSE e incerteza.

In [9]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(0.5, (1e-2, 1)) + WhiteKernel(noise_level=0.5)

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20, random_state=42, alpha=1e-6)
gp.fit(X_train_scaled, y_train)

print("Gaussian Process trained successfully")

Gaussian Process trained successfully


In [10]:
y_pred_test, y_std_test = gp.predict(X_test_scaled, return_std=True)

r2_train = r2_score(y_train, gp.predict(X_train_scaled))
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("="*50)
print("GAUSSIAN PROCESS PERFORMANCE")
print("="*50)
print(f"R² (Train): {r2_train:.4f} | R² (Test): {r2_test:.4f}")
print(f"MAE: {mae_test:.4f}% | RMSE: {rmse_test:.4f}%")
print(f"Mean Uncertainty: {y_std_test.mean():.4f}%")
print("="*50)

GAUSSIAN PROCESS PERFORMANCE
R² (Train): 0.9584 | R² (Test): 0.9651
MAE: 4.9091% | RMSE: 6.5710%
Mean Uncertainty: 7.7987%


## Avaliar performance do modelo

Predições com incerteza no conjunto de teste. Cálculo de R², MAE, RMSE e incerteza média.

## Gerar predições para grid de combinações

Avalia todas as combinações de concentração (0-100%) em passos de 1%. Identifica as top 20 combinações considerando tanto predição quanto incerteza.

In [11]:
best_idx = df[TARGET].argmin()
best_row = df.iloc[best_idx]
best_dmso_obs = best_row['% DMSO']
best_tre_obs = best_row['TREHALOSE']
best_viab_obs = 100 - best_row[TARGET]

print("\n" + "="*50)
print("DATASET BEST OBSERVED")
print("="*50)
print(f"DMSO: {best_dmso_obs:.0f}% | Trehalose: {best_tre_obs:.0f}%")
print(f"Viability: {best_viab_obs:.2f}%")
print()

concentrations = np.arange(0, 101, 1)
grid = np.array(np.meshgrid(concentrations, concentrations)).reshape(2, -1).T
grid_scaled = scaler.transform(grid)

y_grid_pred, y_grid_std = gp.predict(grid_scaled, return_std=True)
y_grid_pred = np.clip(y_grid_pred, 0, 100)

valid_mask = ~((grid[:, 0] == 0) & (grid[:, 1] == 0))
score = y_grid_pred + 0.5 * y_grid_std
score[~valid_mask] = np.inf

optimal_idx = np.argmin(score)
optimal_dmso = grid[optimal_idx, 0]
optimal_tre = grid[optimal_idx, 1]
optimal_pred = y_grid_pred[optimal_idx]
optimal_std = y_grid_std[optimal_idx]

print("="*50)
print("GP MODEL RECOMMENDATION")
print("="*50)
print(f"DMSO: {optimal_dmso:.0f}% | Trehalose: {optimal_tre:.0f}%")
print(f"Predicted Viability: {100 - optimal_pred:.2f}% (±{optimal_std:.2f}%)")
print("="*50)

valid_indices = np.where(valid_mask)[0]
valid_scores = score[valid_indices]
top_20_idx = np.argsort(valid_scores)[:20]
top_20_grid_idx = valid_indices[top_20_idx]


DATASET BEST OBSERVED
DMSO: 2% | Trehalose: 0%
Viability: 99.85%

GP MODEL RECOMMENDATION
DMSO: 18% | Trehalose: 25%
Predicted Viability: 100.00% (±13.99%)


In [12]:
print("\nTOP 20 COMBINATIONS:")
print("-" * 75)
print(f"{'Rank':>4} {'DMSO':>6} {'Trehalose':>10} {'Viability':>12} {'Uncertainty':>12}")
print("-" * 75)
for i, idx in enumerate(top_20_grid_idx, 1):
    dmso_c = grid[idx, 0]
    tre_c = grid[idx, 1]
    viab_drop = y_grid_pred[idx]
    uncertainty = y_grid_std[idx]
    print(f"{i:4d} {dmso_c:5.0f}% {tre_c:9.0f}% {100-viab_drop:11.2f}% {uncertainty:11.2f}%")

print("\n" + "="*75)
print("COMPARISON: GP Prediction vs Dataset Observed")
print("="*75)
print(f"\nDataset Best: DMSO 2% + Trehalose 0% = 99.85% viability (actual)")
print(f"GP Best Rec: DMSO {optimal_dmso:.0f}% + Trehalose {optimal_tre:.0f}% = {100-optimal_pred:.2f}% viability (predicted)")
print(f"             with uncertainty ±{optimal_std:.2f}%")


TOP 20 COMBINATIONS:
---------------------------------------------------------------------------
Rank   DMSO  Trehalose    Viability  Uncertainty
---------------------------------------------------------------------------
   1    18%        25%      100.00%       13.99%
   2     1%         0%       96.88%        7.81%
   3    18%        26%      100.00%       14.05%
   4    18%        27%      100.00%       14.12%
   5     9%         0%       96.71%        7.54%
   6     8%         0%       96.65%        7.54%
   7    18%        24%       99.83%       13.93%
   8    10%         0%       96.59%        7.54%
   9     7%         0%       96.49%        7.53%
  10    18%        28%       99.78%       14.23%
  11     2%         0%       96.43%        7.65%
  12     6%         0%       96.30%        7.52%
  13    19%        23%      100.00%       14.98%
  14    19%        24%      100.00%       15.04%
  15    11%         0%       96.23%        7.54%
  16    19%        25%      100.00%       

## Comparar predição com observações do dataset

Exibição das top 20 combinações e validação com dados observados.